In [49]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.prompts import load_prompt
import json
import time
import datetime
import pypdf

In [50]:
prompt_template = "templates/template.json"
faiss_index = "faiss_index"
data_source = "data/Aboutme"
pdf_source = "data/AM - Resume-.pdf"
pdf_source_1 = "data/AM - Resume1-.pdf"

In [51]:

def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except json.JSONDecodeError:
        return False


In [52]:
import os
os.environ['OPENAI_API_KEY'] = "sk-proj-njseK4HQpzdw4M7QlIi7T3BlbkFJ7g457vzTvs2S1vmcb4Fd"

In [53]:
embeddings = OpenAIEmbeddings()

In [54]:
pdf_loader = PyPDFLoader(pdf_source)
pdf_loader_1 = PyPDFLoader(pdf_source_1)
pdf_data = pdf_loader.load()
pdf_data_1 = pdf_loader.load()

In [55]:
csv_loader = CSVLoader(file_path="data/Aboutme.csv", encoding="utf-8")
csv_data = csv_loader.load()


In [56]:
csv_data

[Document(page_content='what is his educational background| tell me about his education| where did he go to school: does he have any certificates? what kind of certificate he has?\nAbhishek graduates from University of Texas at Dallas in Masters in Business Analytics with majoring in Data Science and Applied Machine Learning. Prior to that, he hold Bachelor of Technology in Digital Communications and Design from Jawaharlal Nehru Technological University: Abhishek is a AWS Certified Solutions Architect and AWS Cloud practitioner. He has a credintial from Alteryx in Machine Learning Fundementals', metadata={'source': 'data/Aboutme.csv', 'row': 0}),
 Document(page_content="what is his educational background| tell me about his education| where did he go to school: How can I contact Abhishek? | What is Dev's email? | How to contact Abhishek?\nAbhishek graduates from University of Texas at Dallas in Masters in Business Analytics with majoring in Data Science and Applied Machine Learning. Pri

In [57]:
# Combine PDF and CSV data
data = pdf_data + csv_data + pdf_data_1

# Create embeddings for the docs
vectors = FAISS.from_documents(data, embeddings)
vectors.save_local("faiss_index")

In [58]:
openai_api_key = os.getenv("sk-proj-njseK4HQpzdw4M7QlIi7T3BlbkFJ7g457vzTvs2S1vmcb4Fd")


In [59]:
prompt = load_prompt("templates/template.json")

In [60]:
retriever = vectors.as_retriever(search_type="similarity", search_kwargs={"k":6, "include_metadata":True, "score_threshold":0.6})
chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(temperature=0.5,model_name='gpt-3.5-turbo', openai_api_key=openai_api_key), 
                                                retriever=retriever,return_source_documents=True,verbose=True,chain_type="stuff",
                                                max_tokens_limit=4097, combine_docs_chain_kwargs={"prompt": prompt})

In [61]:
chat_history = []
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Abhishek Moguluri's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Has Abhishek worked on GenAI?",
                        "chat_history":chat_history})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Resume Bot, a comprehensive, interactive resource for exploring Abhishek's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge. 
 Human: Take a deep breath and do the following step by step these 4 steps:  
 1. Read the context below 
 2. Answer the question with detail using the provided Help Centre information 
 3. Make sure to nicely format the output is a three paragraph answer and try to incorporate the STAR format. 
 4. Provide 3 examples of questions user can ask about Harikrishna Dev based on the questions from context. Context : 
 ~~~ what is his educational background| tell me about his education| where did he go to school: Has Abhishek Worked on Gen AI? | Does Abhishek have experience on Gen AI?
Abhishek graduates from University of Texas at Dallas in Mas

In [62]:
result

{'system': "You are a Resume Bot, a comprehensive, interactive resource for exploring Abhishek Moguluri's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.",
 'question': 'Has Abhishek worked on GenAI?',
 'chat_history': [],
 'answer': '{\n    "answered": true,\n    "response": "Yes, Abhishek has worked in Generative AI. One of his favorite projects was Provoke Hackathon. He also created projects using Open AI, focusing on Emotion Detection and Question Answering.",\n    "questions": [\n        "What are some specific projects Abhishek has worked on in Generative AI?",\n        "Can you elaborate on Abhishek\'s experience with Open AI and its applications in his projects?",\n        "How has Abhishek\'s work in Generative AI contributed to his overall expertise in AI and deep learning?"\n    ]\n}',
 'source_documents': [Document(page_content='what is his educational background| tell m

In [63]:
result['answer']

'{\n    "answered": true,\n    "response": "Yes, Abhishek has worked in Generative AI. One of his favorite projects was Provoke Hackathon. He also created projects using Open AI, focusing on Emotion Detection and Question Answering.",\n    "questions": [\n        "What are some specific projects Abhishek has worked on in Generative AI?",\n        "Can you elaborate on Abhishek\'s experience with Open AI and its applications in his projects?",\n        "How has Abhishek\'s work in Generative AI contributed to his overall expertise in AI and deep learning?"\n    ]\n}'

In [64]:
data = json.loads(result["answer"])

In [65]:
data.get("response")

'Yes, Abhishek has worked in Generative AI. One of his favorite projects was Provoke Hackathon. He also created projects using Open AI, focusing on Emotion Detection and Question Answering.'

In [66]:
data.get("questions")

['What are some specific projects Abhishek has worked on in Generative AI?',
 "Can you elaborate on Abhishek's experience with Open AI and its applications in his projects?",
 "How has Abhishek's work in Generative AI contributed to his overall expertise in AI and deep learning?"]

In [29]:
chat_history.append((result["question"], result["answer"]))

In [67]:
chat_history

[]

In [68]:
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Abhishek's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Can you provide more details about the GenAI application Abhishek worked on?",
                        "chat_history":chat_history})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Resume Bot, a comprehensive, interactive resource for exploring Abhishek's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge. 
 Human: Take a deep breath and do the following step by step these 4 steps:  
 1. Read the context below 
 2. Answer the question with detail using the provided Help Centre information 
 3. Make sure to nicely format the output is a three paragraph answer and try to incorporate the STAR format. 
 4. Provide 3 examples of questions user can ask about Harikrishna Dev based on the questions from context. Context : 
 ~~~ what is his educational background| tell me about his education| where did he go to school: Has Abhishek Worked on Gen AI? | Does Abhishek have experience on Gen AI?
Abhishek graduates from University of Texas at Dallas in Mas

In [69]:
result["answer"]

'{\n    "answered": true,\n    "response": "Abhishek worked on a GenAI application called Provoke Hackathon. His favorite project involved creating an application that matches both images (designer\'s input and developer output) using cosine similarity to generate a visual representation of where the differences are. This project aimed to streamline discrepancies between designer ideas and developer outputs, showcasing his skills in image matching and visual representation.",\n    "questions": [\n        "What other projects has Abhishek worked on related to GenAI?",\n        "Can you elaborate on Abhishek\'s role in the Provoke Hackathon project?",\n        "How did Abhishek\'s GenAI project contribute to improving collaboration between designers and developers?"\n    ]\n}'

In [70]:
chat_history.append((result["question"], result["answer"]))

In [71]:
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Abhishek's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Does he know Snowflake?",
                        "chat_history":chat_history})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Can you provide more details about the GenAI application Abhishek worked on?
Assistant: {
    "answered": true,
    "response": "Abhishek worked on a GenAI application called Provoke Hackathon. His favorite project involved creating an application that matches both images (designer's input and developer output) using cosine similarity to generate a visual representation of where the differences are. This project aimed to streamline discrepancies between designer ideas and developer outputs, showcasing his skills in image matching and visual representation.",
    "questions": [
        "What other projects has Abhishek worked on related to GenAI?",
        "Can you elaborate on Abhishek's role in the Provoke Hackathon project?",
        "How did Abhishek's GenAI

In [72]:
result["answer"]

'{\n  "answered": true,\n  "response": "Abhishek is proficient in Snowflake, a cloud-based data warehousing platform. He has experience working with Snowflake as part of his technical skills.",\n  "questions": [\n    "What other cloud platforms is Abhishek familiar with?",\n    "Can Abhishek provide examples of projects where he used Snowflake?",\n    "How did Abhishek acquire proficiency in Snowflake?"\n  ]\n}'